数据预处理搞错了，把list存进了dataframe里面，结果变成了一个字符串表示的list（假list）。
由于重新预处理太花时间，因此把假list变为list，在变为字符串。

In [0]:
#===从GCS加载数据
import warnings
warnings.filterwarnings("ignore")

project_id = 'amazing-firefly-153908'
bucket_name = 'nlp_final'

# 获取授权
from google.colab import auth
auth.authenticate_user()

#从GCS下载数据至tmp
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')
from apiclient.http import MediaIoBaseDownload

def download_from_GCS(file_name):
  store_path = '/tmp/'+file_name
  with open(store_path, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucket_name,
                                              object=file_name)
    media = MediaIoBaseDownload(f, request)

    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()
  print(file_name + ' downloaded')

In [0]:
from tqdm.auto import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd

import time

In [0]:
def load_data(store_path):
  return pd.read_csv(store_path)

In [0]:
def convert_to_string(data):
  data['question1'] = data['question1'].progress_apply(lambda x:" ".join(eval(x)))
  data['question2'] = data['question2'].progress_apply(lambda x:" ".join(eval(x)))

In [0]:
def store_converted_data(data, suffix, file_name):
  new_file_name = file_name[:-8] + suffix +'.csv'
  new_store_path = '/tmp/' + new_file_name
  data.to_csv(new_store_path, index = False)
  return new_file_name

In [0]:
from googleapiclient.http import MediaFileUpload
def upload_to_GCS(file_name):
  file_path = '/tmp/' + file_name
  
  media = MediaFileUpload(file_path, 
                          mimetype='text/csv',
                          resumable=True)

  request = gcs_service.objects().insert(bucket=bucket_name, 
                                         name=file_name,
                                         media_body=media)

  response = None
  while response is None:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, response = request.next_chunk()
  time_str = get_time()
  print(file_name + ' uploaded at ' + time_str)

In [0]:
def get_time():
  seconds = time.time()
  named_tuple = time.gmtime(seconds) # get struct_time
  hour = named_tuple.tm_hour + 8
  time_str = str(hour) + ':' + str(named_tuple.tm_min) + ':' + str(named_tuple.tm_sec)
  return time_str

In [0]:
def main(file_name):
  store_path = '/tmp/' + file_name
  download_from_GCS(file_name)
  data = load_data(store_path)
  print(file_name + ' read as data')

  time_str = get_time()

  print(file_name + ' convert processe beggin at ' + time_str)
  convert_to_string(data)

  time_str = get_time()
  print('data converted at ' + time_str)

  new_file_name = store_converted_data(data, suffix, file_name)
  print('data stored as ' + new_file_name)
  print('upload' + new_file_name + ' to GCS')
  upload_to_GCS(new_file_name)

In [19]:
file_name = 'x_test_lmtz_cvt.csv'
suffix = '_cvt'
main(file_name)

x_test_lmtz_cvt.csv downloaded
x_test_lmtz_cvt.csv read as data
x_test_lmtz_cvt.csv convert processe beggin at 17:41:28



data converted at 17:43:24
data stored as x_test_lmtz_cvt.csv
uploadx_test_lmtz_cvt.csv to GCS
x_test_lmtz_cvt.csv uploaded at 17:43:44


In [20]:
data = pd.read_csv('/tmp/x_test_lmtz_cvt.csv')
data.head()

,test_id,question1,question2
0,0,how doe the surfac pro himself 4 compar with i...,whi do microsoft choos core m3 and not core i3...
1,1,should i have a hair transplant at age 24 how ...,how much cost doe hair transplant requir
2,2,what but be the best way to send money from ch...,what you send money to china
3,3,which food not emulsifi,what food fibr
4,4,how aberystwyth start read,how their can i start read


In [0]:
files = ['x_test_tk.csv', 'x_test_stm.csv', 'x_test_lmtz.csv'
        , 'x_test_stwd.csv', 'x_test_stwd_stm.csv', 'x_test_stwd_stm.csv']
suffix = '_cvt'

[main(file) for file in files]

x_test_tk.csvdownloaded



progress:   0%|          | 2023/2345796 [00:00<01:55, 20225.49it/s]

x_test_tk.csv read as data
x_test_tk.csv convert processe beggin at 23:34:18



progress:  13%|█▎        | 300301/2345796 [00:07<00:47, 42806.25it/s]